# 线性回归的简洁实现
在过去的几年里，出于对深度学习强烈的兴趣， 许多公司、学者和业余爱好者开发了各种成熟的开源框架。 这些框架可以自动化基于梯度的学习算法中重复性的工作。 在 3.2节中，我们只运用了： （1）通过张量来进行数据存储和线性代数； （2）通过自动微分来计算梯度。 实际上，由于数据迭代器、损失函数、优化器和神经网络层很常用， 现代深度学习库也为我们实现了这些组件。

本节将介绍如何通过使用深度学习框架来简洁地实现 3.2节中的线性回归模型。就不需要我们手写了。

In [15]:
import numpy as np
import tensorflow as tf
import keras
from d2l import tensorflow as d2l

# 生成数据集

In [16]:
true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)
print("y = ", features)
print("x = ", labels)

y =  tf.Tensor(
[[ 0.22045556 -0.5756392 ]
 [-0.04366986 -0.5640035 ]
 [-0.3947528  -1.3582938 ]
 ...
 [-0.6785875   0.3537482 ]
 [-2.0500953  -0.67784756]
 [ 1.3919026   0.61902493]], shape=(1000, 2), dtype=float32)
x =  tf.Tensor(
[[ 6.59509706e+00]
 [ 6.02246523e+00]
 [ 8.03320312e+00]
 [ 3.16176701e+00]
 [ 4.74515200e+00]
 [ 8.54241753e+00]
 [ 7.95159435e+00]
 [ 4.57460928e+00]
 [-7.35723972e-01]
 [ 5.49304628e+00]
 [ 1.13815999e+00]
 [ 2.10677838e+00]
 [-2.63632083e+00]
 [ 5.03679228e+00]
 [ 3.07870865e+00]
 [ 1.04666462e+01]
 [ 2.67249322e+00]
 [-4.17850924e+00]
 [ 1.58719473e+01]
 [ 1.90879834e+00]
 [ 8.03998375e+00]
 [-3.59545851e+00]
 [ 9.55145776e-01]
 [ 2.18164921e+00]
 [ 5.38252640e+00]
 [ 6.62772703e+00]
 [-3.89833021e+00]
 [ 3.13728237e+00]
 [ 5.02453279e+00]
 [ 3.90937001e-01]
 [ 4.38380957e+00]
 [-1.88561630e+00]
 [-1.22933948e+00]
 [ 8.94633865e+00]
 [ 2.83447313e+00]
 [ 3.07284451e+00]
 [ 3.16775608e+00]
 [ 1.07741652e+01]
 [ 1.23524892e+00]
 [ 9.11717129e+00]
 [ 1.07

# 读取数据集

In [17]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个TensorFlow数据迭代器"""
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
    if is_train:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

(<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[-0.8395409 ,  0.01332993],
        [-0.6210282 ,  0.1818055 ],
        [-0.32312623,  0.5743511 ],
        [-0.34194037, -1.8517224 ],
        [-1.132978  ,  0.31187132],
        [ 1.4084982 , -1.1542411 ],
        [ 1.0206444 ,  0.8243155 ],
        [-1.2871473 , -1.6666064 ],
        [ 0.46842214,  0.30643514],
        [-0.07787565, -1.4646736 ]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[ 2.4639688 ],
        [ 2.3433614 ],
        [ 1.6165161 ],
        [ 9.801673  ],
        [ 0.90101236],
        [10.943496  ],
        [ 3.4338195 ],
        [ 7.2805386 ],
        [ 4.1036835 ],
        [ 9.028748  ]], dtype=float32)>)

# 定义模型

In [18]:
net = keras.Sequential()
net.add(keras.layers.Dense(1))

# 初始化模型参数

In [19]:
initializers = tf.random_normal_initializer(stddev=0.01)
net = keras.Sequential()
net.add(keras.layers.Dense(1, kernel_initializer=initializers))

# 定义损失函数，优化算法

In [20]:
loss = keras.losses.MeanSquaredError()
trainer = keras.optimizers.SGD(learning_rate=0.03)

# 训练

In [21]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with tf.GradientTape() as tape:
            l = loss(net(X, training=True), y)
        grads = tape.gradient(l, net.trainable_variables)
        trainer.apply_gradients(zip(grads, net.trainable_variables))
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000278
epoch 2, loss 0.000099
epoch 3, loss 0.000098


In [22]:
w = net.get_weights()[0]
print('w的估计误差：', true_w - tf.reshape(w, true_w.shape))
b = net.get_weights()[1]
print('b的估计误差：', true_b - b)

w的估计误差： tf.Tensor([-0.00024629 -0.00028992], shape=(2,), dtype=float32)
b的估计误差： [0.00013256]
